In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from torch.utils.data import DataLoader

from utils import create_save_folder, EarlyStopping

os.environ["CUDA_VISIBLE_DEVICES"]="0"

fm_path = create_save_folder()

In [2]:
class denoising_model(nn.Module):
    def __init__(self, num_columns):
        super(denoising_model,self).__init__()
        self.encoder=nn.Sequential(
            nn.Linear(num_columns,256),
            # nn.BatchNorm1d(256),
            nn.SiLU(True),
            nn.Linear(256,128),
            # nn.BatchNorm1d(128),
            nn.SiLU(True),
        )
        
        self.decoder=nn.Sequential(
            nn.Linear(128,256),
            # nn.BatchNorm1d(256),
            nn.SiLU(True),
            nn.Linear(256, num_columns),
            # nn.BatchNorm1d(num_columns),
            nn.SiLU(True),
        )

    def forward(self,x):
        x=self.encoder(x)
        x=self.decoder(x)
        
        return x
    
    def encode(self, x):
        return self.encoder(x)

In [3]:
class DataSet:
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # noise = torch.randn(self.data.shape[1]).cuda()
        # clean = self.data[index]
        # dirty = self.data[index] + noise
        # return clean, dirty
        return self.data[index]


In [4]:
with open('/home/yoshikawa/work/kaggle/OPVP/output/feature_model/20210824/0/train.pkl', 'rb') as f:
    df_train = pickle.load(f)

In [5]:
train = df_train.drop(['row_id', 'target'], axis=1)
for col in train.columns.to_list():
    train[col] = train[col].fillna(train[col].mean())

scales = train.drop(["stock_id"], axis = 1).columns.to_list()

scaler = StandardScaler()
scaler.fit(train[scales])
train[scales] = scaler.transform(train[scales])
le = LabelEncoder()
le.fit(train["stock_id"])
train["stock_id"] = le.transform(train["stock_id"])

In [6]:
train_data = torch.tensor(train.values.astype(np.float32)).cuda()
train_data.shape[1]

230

In [12]:
criterion = nn.MSELoss()
kf = KFold(n_splits=5, shuffle=True, random_state=55)
epochs = 10000

cv = 0
models = []
for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
    print('fold: ', fold)
    print('='*100)
    train_dataset = DataSet(train_data[train_idx].cuda())
    val_dataset = DataSet(train_data[val_idx].cuda())
    train_loader = DataLoader(train_dataset, 4096, shuffle=True)
    val_loader = DataLoader(val_dataset, 4096)
    
    model = denoising_model(train_data.shape[1]).cuda()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    earlystopping = EarlyStopping(patience=10, verbose=True, path=fm_path+'/checkpoint.pth')

    for i in range(epochs):
        model.train()
        train_loss, val_loss = 0, 0

        for j, data in enumerate(train_loader):
            noise = torch.randn(data.shape).cuda()
            dirty = data + noise
            output = model(dirty)
            loss = criterion(output, data)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.shape[0]
        
        train_loss /= len(train_dataset)
        
        for j, data in enumerate(val_loader):
            noise = torch.randn(data.shape).cuda()
            dirty = data + noise
            output = model(dirty)
            loss = criterion(output, data)
            val_loss += loss.item() * data.shape[0]
        
        val_loss /= len(val_dataset)
        if (i+1) % 10 == 0:
            print(i+1, " epoch - train_loss: ", round(train_loss, 4), ", val_loss: ", round(val_loss, 4))
        earlystopping(val_loss, model)
        if earlystopping.early_stop:
            print("Early Stopping!!")
            break
    cv += val_loss
    model.load_state_dict(torch.load(fm_path+'/checkpoint.pth'))
    models.append(model)
cv /= 5
print("cv: ", round(cv, 4))

fold:  0
Validation loss decreased (inf --> 0.426466).  Saving model ...
Validation loss decreased (0.426466 --> 0.341160).  Saving model ...
Validation loss decreased (0.341160 --> 0.304301).  Saving model ...
Validation loss decreased (0.304301 --> 0.287294).  Saving model ...
Validation loss decreased (0.287294 --> 0.279119).  Saving model ...
Validation loss decreased (0.279119 --> 0.274693).  Saving model ...
Validation loss decreased (0.274693 --> 0.269382).  Saving model ...
Validation loss decreased (0.269382 --> 0.261899).  Saving model ...
Validation loss decreased (0.261899 --> 0.259224).  Saving model ...
10  epoch - train_loss:  0.2604 , val_loss:  0.2558
Validation loss decreased (0.259224 --> 0.255815).  Saving model ...
Validation loss decreased (0.255815 --> 0.254173).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.254173 --> 0.249711).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.249711 --> 0.2468

In [13]:
output = torch.zeros((train_data.shape[0], 128))
for i, model in enumerate(models):
    # train_dataset = DataSet(train_data)
    # train_loader = DataLoader(train_dataset, 4096, shuffle=False)
    # for j, data in enumerate(train_loader):
    output += model.encode(train_data).cpu() / 5
    torch.save(model.state_dict(), fm_path+'/model-'+str(i))


In [14]:
output

tensor([[1.4571, 1.2547, 1.9901,  ..., 2.6923, 1.7736, 0.6860],
        [1.3585, 0.8547, 1.8584,  ..., 2.7255, 1.8111, 0.5490],
        [1.3632, 1.1603, 2.0882,  ..., 2.9494, 1.4200, 1.2114],
        ...,
        [3.2910, 1.8508, 4.6874,  ..., 2.9970, 3.4341, 2.3833],
        [3.0671, 1.9353, 4.5361,  ..., 2.8554, 2.9525, 2.6529],
        [3.1853, 1.4961, 4.5273,  ..., 2.9214, 3.0446, 2.6488]],
       grad_fn=<AddBackward0>)

In [15]:
output.shape

torch.Size([428932, 128])

In [16]:
df_output = pd.DataFrame(output.detach().numpy())
df_output.columns = ['DAE_'+ str(i) for i in df_output.columns]
df_output.describe()

,DAE_0,DAE_1,DAE_2,DAE_3,DAE_4,DAE_5,DAE_6,DAE_7,DAE_8,DAE_9,DAE_10,DAE_11,DAE_12,DAE_13,DAE_14,DAE_15,DAE_16,DAE_17,DAE_18,DAE_19,DAE_20,DAE_21,DAE_22,DAE_23,DAE_24,DAE_25,DAE_26,DAE_27,DAE_28,DAE_29,DAE_30,DAE_31,DAE_32,DAE_33,DAE_34,DAE_35,DAE_36,DAE_37,DAE_38,DAE_39,DAE_40,DAE_41,DAE_42,DAE_43,DAE_44,DAE_45,DAE_46,DAE_47,DAE_48,DAE_49,DAE_50,DAE_51,DAE_52,DAE_53,DAE_54,DAE_55,DAE_56,DAE_57,DAE_58,DAE_59,DAE_60,DAE_61,DAE_62,DAE_63,DAE_64,DAE_65,DAE_66,DAE_67,DAE_68,DAE_69,DAE_70,DAE_71,DAE_72,DAE_73,DAE_74,DAE_75,DAE_76,DAE_77,DAE_78,DAE_79,DAE_80,DAE_81,DAE_82,DAE_83,DAE_84,DAE_85,DAE_86,DAE_87,DAE_88,DAE_89,DAE_90,DAE_91,DAE_92,DAE_93,DAE_94,DAE_95,DAE_96,DAE_97,DAE_98,DAE_99,DAE_100,DAE_101,DAE_102,DAE_103,DAE_104,DAE_105,DAE_106,DAE_107,DAE_108,DAE_109,DAE_110,DAE_111,DAE_112,DAE_113,DAE_114,DAE_115,DAE_116,DAE_117,DAE_118,DAE_119,DAE_120,DAE_121,DAE_122,DAE_123,DAE_124,DAE_125,DAE_126,DAE_127
count,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000,428932.000000
mean,2.024674,1.408132,2.937426,2.922235,1.404105,2.793133,2.509424,0.836562,3.257326,3.626128,1.956815,1.648261,2.811153,3.208830,1.484894,2.376671,3.860445,1.679050,1.981682,1.661340,2.739988,1.137162,2.801373,2.568943,3.491256,4.054435,1.839545,2.412157,2.417036,0.990277,0.898654,1.914059,2.411544,2.159241,2.735686,2.314098,1.951012,1.348031,3.207363,2.653598,1.316775,3.112838,1.679902,3.475783,2.062476,0.473169,0.911093,2.108604,1.854478,2.301869,3.907174,2.412498,2.093194,2.144174,2.688028,2.562392,0.542321,2.804337,4.070835,2.631412,3.579354,0.839137,2.388687,3.143724,1.632429,2.864942,5.138174,3.223232,1.306428,3.053071,0.973733,1.451537,0.411263,2.231162,2.314613,2.731990,1.031546,2.230681,3.490454,1.545617,2.844125,2.217495,2.502552,0.874606,1.035058,4.537579,1.466348,2.935630,2.836405,1.596715,2.341712,1.807350,2.057317,2.665547,3.989386,3.086629,3.927931,1.440984,2.123539,1.637443,2.927938,2.510748,1.107941,3.701484,4.635833,0.525721,1.134774,2.043727,3.750197,3.535721,1.566113,1.331328,2.827906,1.259785,3.121304,1.359910,1.340247,3.293372,4.014331,2.090719,1.842517,1.283807,3.295710,2.128874,2.004336,2.287117,1.753305,1.867003
std,1.017644,0.951307,1.306224,1.491263,0.769466,1.364367,1.119864,0.786350,1.459839,1.641880,1.332849,0.882851,1.600111,1.297801,

In [19]:
df = pd.concat([df_train, df_output], axis=1)
pickle.dump(df, open(os.path.join(fm_path, "train.pkl"), 'wb'))